In [1]:
import warnings

import torch

from omnisafe.common.experiment_grid import ExperimentGrid
import omnisafe

In [5]:
eg = ExperimentGrid(exp_name='BM_Safety_CPO')

second_order_policy = ['CPO']

safety_envs = [
    'SafetyPointGoal1-v0',
]
eg.add('env_id', safety_envs)

avaliable_gpus = list(range(torch.cuda.device_count()))
gpu_id = [0]

if gpu_id and not set(gpu_id).issubset(avaliable_gpus):
    warnings.warn('The GPU ID is not available, use CPU instead.', stacklevel=1)
    gpu_id = None

eg.add('algo', second_order_policy)
eg.add('logger_cfgs:use_wandb', [True])
eg.add('logger_cfgs:wandb_project', ['Benchmark_Safety'])
eg.add('train_cfgs:vector_env_nums', [4])
eg.add('train_cfgs:torch_threads', [8])
eg.add('algo_cfgs:steps_per_epoch', [20000])
eg.add('train_cfgs:total_steps', [20000 * 30])
eg.add('seed', [0])

In [6]:
"""Example of training a policy from exp-x config with OmniSafe."""
import os, sys
from omnisafe.typing import NamedTuple, Tuple


def train(
    exp_id: str, algo: str, env_id: str, custom_cfgs: NamedTuple
) -> Tuple[float, float, float]:
    """Train a policy from exp-x config with OmniSafe.

    Args:
        exp_id (str): Experiment ID.
        algo (str): Algorithm to train.
        env_id (str): The name of test environment.
        custom_cfgs (NamedTuple): Custom configurations.
        num_threads (int, optional): Number of threads. Defaults to 6.
    """
    terminal_log_name = 'terminal.log'
    error_log_name = 'error.log'
    if 'seed' in custom_cfgs:
        terminal_log_name = f'seed{custom_cfgs["seed"]}_{terminal_log_name}'
        error_log_name = f'seed{custom_cfgs["seed"]}_{error_log_name}'
    sys.stdout = sys.__stdout__
    sys.stderr = sys.__stderr__
    print(f'exp-x: {exp_id} is training...')
    if not os.path.exists(custom_cfgs['logger_cfgs']['log_dir']):
        os.makedirs(custom_cfgs['logger_cfgs']['log_dir'], exist_ok=True)
    # pylint: disable-next=consider-using-with
    sys.stdout = open(
        os.path.join(f'{custom_cfgs["logger_cfgs"]["log_dir"]}', terminal_log_name),
        'w',
        encoding='utf-8',
    )
    # pylint: disable-next=consider-using-with
    sys.stderr = open(
        os.path.join(f'{custom_cfgs["logger_cfgs"]["log_dir"]}', error_log_name),
        'w',
        encoding='utf-8',
    )
    agent = omnisafe.Agent(algo, env_id, custom_cfgs=custom_cfgs)
    reward, cost, ep_len = agent.learn()
    return reward, cost, ep_len

In [7]:
# total experiment num must can be divided by num_pool
# meanwhile, users should decide this value according to their machine
eg.run(train, gpu_id=gpu_id)

Save with config of experiment grid in grid_config.json

ExperimentGrid [BM_Safety_CPO] runs over parameters:

 env_id                                  

[env] 

	"SafetyPointGoal1-v0"



 algo                                    

...

Logging data to 
./exp-x\BM_Safety_CPO\SafetyPointGoal1-v0---a666ef11f41efb3aebf109cd70c4d4f6f3fe654ce38f5144dfb4fe07a0c7c165\CPO-{S
afetyPointGoal1-v0}\seed-000-2024-04-24-11-29-43\progress.csv

Save with config in config.json

project Benchmark_Safety name CPO-{SafetyPointGoal1-v0}-seed-000-2024-04-24-11-29-43

INFO: Start training

c:\Users\saisa\.conda\envs\omnisafe-env\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [ ]:
LOG_DIR = 'exp-x\BM_Safety_CPO\SafetyPointGoal1-v0---a666ef11f41efb3aebf109cd70c4d4f6f3fe654ce38f5144dfb4fe07a0c7c165\CPO-{SafetyPointGoal1-v0}\seed-000-2024-04-24-11-29-43'

evaluator = omnisafe.Evaluator()
for item in os.scandir(os.path.join(LOG_DIR, 'torch_save')):
    if item.is_file() and item.name.split('.')[-1] == 'pt':
        evaluator.load_saved(
            save_dir=LOG_DIR, model_name=item.name, camera_name='track', width=256, height=256
        )
        evaluator.render(num_episodes=5)
        evaluator.evaluate(num_episodes=5)